In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyarrow.parquet as pq

In [9]:
with open("Beneficiarios Innominada 2023/Beneficiarios Fonasa 2023.csv", encoding='utf-8', errors='ignore') as f:
    df = pd.read_csv(f,nrows=10**6 * 3)

In [10]:
df.shape

(3000000, 12)

In [11]:
with open("Beneficiarios Innominada 2023/Beneficiarios Fonasa 2023.csv", encoding='utf-8', errors='ignore') as f:
    df2 = pd.read_csv(f, nrows=1000)

In [12]:
df.columns

Index(['MES_INFORMACION', 'CDIGO_BENEFICIARIO', 'SEXO', 'EDAD_TRAMO',
       'TITULAR_CARGA', 'TRAMO_FONASA', 'TRAMO_RENTA', 'NACIONALIDAD',
       'CARACTERIZACIN', 'TIPO_ASEGURADO', 'REGIN_BENEFICIARIO',
       'COMUNA_BENEFICIARIO'],
      dtype='object')

In [12]:
df2.sample(n=10)

,MES_INFORMACION,CDIGO_BENEFICIARIO,SEXO,EDAD_TRAMO,TITULAR_CARGA,TRAMO_FONASA,TRAMO_RENTA,NACIONALIDAD,CARACTERIZACIN,TIPO_ASEGURADO,REGIN_BENEFICIARIO,COMUNA_BENEFICIARIO
234,202312,84807171,Hombre,30 a 39 aos,Titular,B,Sin Cotizaciones,Chilena,Independiente,Trabajador Independiente,Del Maule,Talca
858,202312,88200367,Hombre,20 a 29 aos,Titular,D,500.001 - 550.000,Chilena,Dependiente,Trabajador Dependiente,Desconocida,Desconocida
339,202312,86109888,Mujer,30 a 39 aos,Titular,D,750.001 - 800.000,Chilena,Dependiente,Trabajador Dependiente,De Los Lagos,Puerto Varas
790,202312,78079304,Hombre,20 a 29 aos,Titular,D,500.001 - 550.000,Chilena,Dependiente,Trabajador Dependiente,De Los Lagos,Puerto Montt
335,202312,69584385,Mujer,30 a 39 aos,Titular,A,Sin Cotizaciones,Chilena,Suf,Carente,Del Maule,Longav
298,202312,76620946,Mujer,30 a 39 aos,Titular,B,300.001 - 350.000,Chilena,Independiente,Trabajador Dependiente,Metropolitana De Santiago,Pealoln
923,202312,81283518,Hombre,20 a 29 aos,Titular,D,mayor a 1.600.00,Chilena,Dependiente,Trabajador Dependiente,De Coquimbo,La Serena
102,202312,69560751,Hombre,30 a 39 aos,Titular,D,1.450.001 - 1.500.000,Chilena,Dependiente,Trabajador Dependiente,De Coquimbo,Ovalle
90,202312,83167722,Mujer,30 a 39 aos,Titular,C,300.001 - 350.000,Chilena,Desempleado Con Seguro De Desempleo,Trabajador Dependiente,Metropolitana De Santiago,Pedro Aguirre Cerda
73,202312,67664426,Mujer,40 a 49 aos,Titular,C,400.001 - 450.000,Chilena,Dependiente,Trabajador Dependiente,De Valparaso,Casablanca


In [7]:
def imprimir_filas(df):
    for index, fila in df.iterrows():
        print(f"Fila {index}:")
        print(fila)
        print("-" * 20)

imprimir_filas(df)